In [1]:
import pandas as pd 
import numpy as np

import json
import sys
from pathlib import Path

proj_path = Path('/cluster') / 'work' / 'stefandt' / 'pers-pred'
resolved = proj_path.resolve()
if resolved not in sys.path: sys.path.append(str(resolved))

from src.utils import get_commons
# from src.datasets import split_imputed_dataframe

paths, constants, config, logger, device = get_commons()

2024-06-23 10:08:39,548 - ArgumentLogger - INFO - Arguments:
2024-06-23 10:08:39,551 - ArgumentLogger - INFO - seed: 42
2024-06-23 10:08:39,553 - ArgumentLogger - INFO - dataframe: {'generate': False, 'mbti_frac': 0.1, 'bigfive_c_frac': 1.0, 'bigfive_s_frac': 1.0}
2024-06-23 10:08:39,554 - ArgumentLogger - INFO - eda: {'generate': False}
2024-06-23 10:08:39,555 - ArgumentLogger - INFO - reduce: {'generate': False, 'use_full': False}
2024-06-23 10:08:39,556 - ArgumentLogger - INFO - preprocessing: {'generate_features': False, 'generate_partially_cleaned': False, 'generate_cleaned': False, 'generate_embeddings': False, 'generate_aggregated': True, 'generate_glove': False, 'generate_filled': True}
2024-06-23 10:08:39,557 - ArgumentLogger - INFO - imputation: {'num_of_clusters': 4, 'n_components': 2}
2024-06-23 10:08:39,558 - ArgumentLogger - INFO - split: {'train': 0.8, 'test': 0.1}
2024-06-23 10:08:39,558 - ArgumentLogger - INFO - dataloaders: {'train': {'num_workers': 1, 'pin_memory': F

device: cpu


In [20]:
emb = pd.read_csv(paths['new']['embeddings']['xlnet'], index_col=0)

In [21]:
emb

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
-1000426711057652199,0.137424,2.576350,-1.854247,0.298307,-1.068546,-2.754340,-0.867802,-1.511619,-0.834488,-1.062465,...,1.367905,0.851196,-3.045871,-1.884005,0.143856,-4.667974,1.119782,0.434197,0.167776,-0.694719
-1000672945006441368,-1.110359,2.072212,-1.119827,0.847412,-1.227310,-2.083271,-1.208566,-0.905945,-0.926216,-0.804096,...,1.038508,1.146372,-2.705535,-1.188822,0.507912,-4.090681,0.354466,0.485762,0.695365,-0.677387
-1001113311319652785,-0.127591,-0.108305,-0.630968,1.120523,-0.302309,-0.365912,1.302331,0.280672,-0.740595,1.150290,...,-0.329067,0.635081,0.548500,1.033785,1.903216,-1.810187,-1.644113,0.091589,1.390022,0.172862
-1001525054643540721,-0.713222,1.955955,-1.297772,0.887470,-1.249359,-2.568797,-0.922795,-0.776647,-1.171110,-0.415578,...,0.774727,1.123784,-2.573004,-1.266037,0.402762,-4.179205,0.264898,0.345798,0.476109,-0.256281
-1001584773999252365,-1.170908,2.163388,-1.233108,1.142720,-1.102421,-2.077665,-1.043161,-0.591217,-0.538860,-0.638723,...,1.525123,1.303941,-2.471210,-0.735072,0.250753,-3.747939,0.371207,0.561219,0.727129,-0.660567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997572341142053346,-0.263388,1.803279,-1.594797,0.056632,-1.290793,-1.830619,-0.673220,-0.723112,-1.035746,-0.185205,...,0.871584,0.074158,-2.390033,-1.784003,0.210363,-5.714864,1.230967,-0.610375,0.310579,-0.375450
998744985101306774,-0.884572,2.143137,-1.201241,0.757430,-1.280775,-2.308861,-1.082980,-0.924514,-0.942491,-0.789748,...,1.001363,1.164388,-2.661016,-1.281233,0.357350,-4.389536,0.639599,0.578924,0.594778,-0.630654
999571443209289003,0.007625,1.812995,-1.518418,0.048562,-0.832213,-2.920786,-0.290583,-0.913227,-1.311068,-0.341111,...,0.569202,-0.051025,-2.613598,-1.996052,0.591099,-5.748655,0.918397,-0.296931,0.342893,-0.169144
999590181235062715,-0.817302,2.081244,-1.580119,0.222159,-1.603494,-1.513610,-1.104035,0.110908,-0.885838,-0.362920,...,1.129293,0.892704,-2.086449,-1.455747,0.069524,-4.934519,1.023201,-0.196793,0.098123,-0.610968


In [8]:
emb

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
-1000426711057652199,-0.086843,-0.092238,-0.059126,-0.144407,-0.108827,-0.177840,0.214415,0.357788,-0.072667,-0.249636,...,0.024953,-0.179545,0.087047,-0.143213,0.154646,0.021004,-0.054539,-0.078015,0.368330,0.341988
-1000672945006441368,0.037238,-0.043906,-0.069089,-0.131858,-0.100645,-0.208742,0.211818,0.374280,-0.097693,-0.246682,...,-0.021515,-0.223710,0.084120,-0.152658,0.135765,0.084989,-0.104788,-0.045634,0.355514,0.339951
-1001113311319652785,0.034650,0.048038,-0.176649,-0.334682,-0.184278,-0.075854,0.499212,0.462911,-0.180647,-0.263625,...,0.037797,-0.352404,0.207580,-0.085410,0.307661,0.177866,-0.083783,0.208709,0.217568,0.204969
-1001525054643540721,0.062761,-0.032273,-0.065706,-0.120092,-0.077418,-0.157687,0.236620,0.396462,-0.123410,-0.220317,...,-0.024286,-0.212614,0.043939,-0.182105,0.141383,0.071966,-0.142136,-0.080675,0.391070,0.333112
-1001584773999252365,0.008426,-0.050172,-0.099671,-0.142722,-0.119701,-0.150644,0.255862,0.328549,-0.095449,-0.240582,...,-0.057180,-0.205757,0.080162,-0.148111,0.127359,0.041881,-0.110017,-0.055660,0.377419,0.333747
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
997572341142053346,-0.171817,0.012905,-0.079998,-0.302956,-0.237839,-0.098261,0.280103,0.249433,-0.193503,-0.166305,...,0.155508,-0.182655,0.097161,-0.111193,0.062535,-0.074487,-0.159729,-0.062283,0.248569,0.287186
998744985101306774,0.044872,0.014799,-0.052564,-0.162829,-0.100329,-0.172381,0.224394,0.395496,-0.115006,-0.253343,...,0.000926,-0.224102,0.085503,-0.172430,0.157380,0.053551,-0.132112,-0.054823,0.357492,0.288657
999571443209289003,-0.132367,-0.007094,0.024896,-0.235788,-0.219715,-0.156387,0.303409,0.345938,-0.141390,-0.214914,...,0.119320,-0.229736,0.111446,-0.113909,0.160593,-0.057810,-0.078694,-0.090819,0.291665,0.329799
999590181235062715,-0.138389,-0.029851,-0.060207,-0.277761,-0.238060,-0.087106,0.325941,0.364436,-0.203054,-0.219948,...,0.144178,-0.161481,0.031225,-0.154045,0.109910,-0.008503,-0.134395,-0.089831,0.257271,0.276761


In [9]:
old_emb = pd.read_csv(paths['new']['old_embeddings']['distilbert'], index_col=0)

In [19]:
old_emb.loc[999927629210385999]

KeyError: 999927629210385999

In [4]:
results = pd.read_csv(paths['new']['split']['bigfive_s'], header=[0, 1], index_col=0)

In [6]:
results['ORG_TARGET']

FEATURE,sEXT,sNEU,sAGR,sCON,sOPN
AUTHOR,,,,,
-9223192462986286358,NaN,NaN,NaN,NaN,NaN
-9223027331140286803,NaN,NaN,NaN,NaN,NaN
-9222856035137654294,NaN,NaN,NaN,NaN,NaN
-9222804379051090963,NaN,NaN,NaN,NaN,NaN
-9222547355321442980,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...
9220025367096517589,NaN,NaN,NaN,NaN,NaN
9221137650467601654,NaN,NaN,NaN,NaN,NaN
9221253244137085517,NaN,NaN,NaN,NaN,NaN


In [10]:
results['ORG_TARGET'].dropna(axis=0)

FEATURE,sEXT,sNEU,sAGR,sCON,sOPN
AUTHOR,,,,,
-9209709594471780623,50.0,50.0,30.0,50.0,85.0
-9189368714324501202,37.0,36.0,15.0,66.0,68.0
-9179932949514489112,14.0,17.0,41.0,44.0,43.0
-9178256801304560551,1.0,85.0,57.0,28.0,96.0
-9149934999937013887,22.0,90.0,45.0,13.0,64.0
...,...,...,...,...,...
9153902383019991685,58.0,58.0,89.0,97.0,84.0
9166283747925854102,22.0,19.0,34.0,80.0,26.0
9170849315274659739,54.0,32.6,55.0,37.4,11.6


In [14]:
results['EW']['best_result']['result']

{'mbti': [0.336022287607193],
 'bigfive_c': [0.5321129361788431],
 'bigfive_s': [44.69022579445188]}

In [11]:

df = pd.DataFrame(results, index=["best_result", "total_time"])
# Function to unpack the JSON
def unpack_json(row):
    new_row = {}
    for col in row.index:
        try:
            new_row[col] = json.loads(row[col].replace("'", '"'))
        except:
            new_row[col] = row[col]
    return pd.Series(new_row)

# Apply the function to unpack the JSON
df.loc["best_result"] = unpack_json(df.loc["best_result"])

# Create a new dataframe from the unpacked JSON
unpacked_df = pd.json_normalize(df.loc["best_result"])

# Concatenate the unpacked dataframe with the total_time row
final_df = pd.concat([unpacked_df, df.loc[["total_time"]].reset_index(drop=True)], axis=1)

# Display the final dataframe
print(final_df.columns)

Index(['improvement', 'epoch', 'result.mbti', 'result.bigfive_c',
       'result.bigfive_s', 'EW', 'GradNorm', 'MGDA', 'UW', 'DWA', 'GLS',
       'GradDrop', 'IMTL', 'RLW', 'Aligned_MTL'],
      dtype='object')


In [25]:
emb = pd.read_csv(paths['new']['embeddings']['xlnet'], index_col=0, nrows=10)

In [26]:
emb.head()

,0,1,2,3,4,5,6,7,8,9,...,758,759,760,761,762,763,764,765,766,767
-1000743518259762586,-1.517252,2.090348,-1.104183,0.726501,-1.226219,-1.499407,-1.150750,-0.347007,-0.613371,-0.482898,...,1.139784,1.386529,-2.199995,-0.482462,0.405805,-2.974739,0.350724,0.321031,1.069689,-0.677038
-1002088869937924482,-0.803348,1.999382,-1.023048,0.763030,-1.284239,-2.387395,-1.276879,-1.260743,-0.875587,-0.576327,...,0.869426,1.111919,-2.772670,-1.213847,0.316339,-4.663496,0.560791,0.385378,1.016323,-0.532036
-1003169028402896436,-0.745728,1.621405,-0.922467,0.813447,-1.337052,-2.601058,-1.272724,-0.688592,-1.790440,-0.357332,...,0.652398,0.368152,-2.885623,-1.880712,0.376326,-5.171672,1.005808,0.316924,0.299690,-0.249223
-1003226633394130885,-1.210843,1.384257,-0.978206,0.912521,-0.850152,-0.970951,-0.552447,0.047600,-0.272277,-0.204958,...,1.118028,1.224283,-1.352433,-0.108125,0.261877,-2.837660,-0.143890,0.097265,0.644296,-0.707609
-100365023987329479,-1.096222,1.880861,-1.104189,0.725087,-1.198385,-1.954089,-1.132847,-0.938533,-0.688848,-0.621122,...,1.179656,1.192853,-2.623702,-1.182995,0.311249,-4.123425,0.570933,0.462102,0.952408,-0.698596


In [2]:
df = pd.read_csv(paths['new']['imputed-all'], header=[0, 1], index_col=0)

In [3]:
len(df['STATS'].columns)

12

In [4]:
targets = df[[('ORG_TARGET', col) for col in constants['target_columns']]]


In [6]:
targets.head()

GROUP                ORG_TARGET                                              \
FEATURE                 mbtiEXT mbtiSEN mbtiTHI mbtiJUD cEXT cNEU cAGR cCON   
AUTHOR                                                                        
-9223192462986286358        0.0     0.0     0.0     1.0  NaN  NaN  NaN  NaN   
-9223027331140286803        0.0     0.0     0.0     0.0  NaN  NaN  NaN  NaN   
-9222856035137654294        0.0     0.0     0.0     0.0  NaN  NaN  NaN  NaN   
-9222804379051090963        0.0     0.0     0.0     1.0  NaN  NaN  NaN  NaN   
-9222547355321442980        0.0     0.0     0.0     1.0  NaN  NaN  NaN  NaN   

GROUP                                               
FEATURE              cOPN sEXT sNEU sAGR sCON sOPN  
AUTHOR                                              
-9223192462986286358  NaN  NaN  NaN  NaN  NaN  NaN  
-9223027331140286803  NaN  NaN  NaN  NaN  NaN  NaN  
-9222856035137654294  NaN  NaN  NaN  NaN  NaN  NaN  
-9222804379051090963  NaN  NaN  NaN  NaN  NaN  NaN  
-9222547355321442980  NaN  NaN  NaN  NaN  NaN  NaN

In [7]:
targets.columns = targets.columns.droplevel(0) 

In [8]:
targets[constants['mbti_columns']] = targets[constants['mbti_columns']].astype(bool)
targets[constants['bigfive_c_columns']] = targets[constants['bigfive_c_columns']].astype(bool)

/tmp/ipykernel_164648/2527210562.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets[constants['mbti_columns']] = targets[constants['mbti_columns']].astype(bool)
/tmp/ipykernel_164648/2527210562.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  targets[constants['bigfive_c_columns']] = targets[constants['bigfive_c_columns']].astype(bool)


In [9]:
targets

FEATURE,mbtiEXT,mbtiSEN,mbtiTHI,mbtiJUD,cEXT,cNEU,cAGR,cCON,cOPN,sEXT,sNEU,sAGR,sCON,sOPN
AUTHOR,,,,,,,,,,,,,,
-9223192462986286358,False,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9223027331140286803,False,False,False,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9222856035137654294,False,False,False,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9222804379051090963,False,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9222547355321442980,False,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9220025367096517589,False,False,True,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
9221137650467601654,True,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
9221253244137085517,True,False,False,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN


In [12]:
targets.loc[targets['mbtiEXT'] != targets['cEXT']]

FEATURE,mbtiEXT,mbtiSEN,mbtiTHI,mbtiJUD,cEXT,cNEU,cAGR,cCON,cOPN,sEXT,sNEU,sAGR,sCON,sOPN
AUTHOR,,,,,,,,,,,,,,
-9223192462986286358,False,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9223027331140286803,False,False,False,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9222856035137654294,False,False,False,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9222804379051090963,False,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
-9222547355321442980,False,False,False,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9217209805049103980,False,False,False,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
9217577976937101211,False,False,True,False,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN
9218265386908415021,False,True,True,True,True,True,True,True,True,NaN,NaN,NaN,NaN,NaN


In [ ]:
targets.loc[target[]]

In [3]:
rng = np.random.default_rng(42)

In [5]:
df

GROUP                      STATS                                          \
FEATURE                    chars uppercased    emojis   posts duplicates   
AUTHOR                                                                     
-9223192462986286358   67.496063   4.929134  0.850394   127.0   0.157480   
-9223027331140286803  169.280000   6.600000  0.000000    50.0   0.840000   
-9222856035137654294  122.727273   6.636364  0.000000    33.0   0.333333   
-9222804379051090963  157.413043   6.956522  0.000000    46.0   0.934783   
-9222547355321442980  154.255319   5.063830  0.000000    47.0   0.744681   
...                          ...        ...       ...     ...        ...   
 9220025367096517589   68.147368  13.684211  0.957895    95.0   0.357895   
 9221137650467601654  164.494331   5.718821  0.000378  2646.0   0.134921   
 9221253244137085517  140.621622   3.081081  0.000000    37.0   0.783784   
 9222615797454098519  236.614375   9.807286  0.003917  5106.0   0.135527   
 9223299406724851482   84.261146  16.127389  2.528662   157.0   0.509554   

GROUP                                                            \
FEATURE              word_nonwords nonword_words nonword_spaces   
AUTHOR                                                            
-9223192462986286358      2.992126      2.212598       0.622047   
-9223027331140286803      5.680000      1.940000       1.460000   
-9222856035137654294      4.060606      1.363636       0.545455   
-9222804379051090963      5.108696      1.847826       0.717391   
-9222547355321442980      5.255319      1.872340       0.191489   
...                            ...           ...            ...   
 9220025367096517589      2.484211      2.589474       0.136842   
 9221137650467601654      5.421013      2.487150       0.082011   
 9221253244137085517      3.270270      1.270270       0.216216   
 9222615797454098519      8.597336      4.169800       0.356639   
 9223299406724851482      2.605096      2.840764       0.121019   

GROUP                                              ... IMP_TARGET            \
FEATURE              space_punctuations  hashtags  ...       cEXT cNEU cAGR   
AUTHOR                                             ...                        
-9223192462986286358           0.007874  0.299213  ...          0    1    0   
-9223027331140286803           0.200000  0.000000  ...          0    1    0   
-9222856035137654294           0.000000  0.000000  ...          0    1    0   
-9222804379051090963           0.195652  0.000000  ...          0    1    0   
-9222547355321442980           0.000000  0.000000  ...          0    1    0   
...                                 ...       ...  ...        ...  ...  ...   
 9220025367096517589           0.021053  0.189474  ...          0    0    0   
 9221137650467601654           0.001512  0.002268  ...          1    1    0   
 9221253244137085517           0.027027  0.027027  ...          1    1    1   
 9222615797454098519           0.000979  0.001175  ...          1    1    1   
 9223299406724851482           0.000000  0.000000  ...          0    1    0   

GROUP                                                                       \
FEATURE              cCON cOPN       sEXT       sNEU       sAGR       sCON   
AUTHOR                                                                       
-9223192462986286358    0    1  34.366476  53.679924  46.349121  45.043467   
-9223027331140286803    0    1  33.464882  51.631764  49.345049  35.043664   
-9222856035137654294    0    1  33.464882  51.631764  49.345049  35.043664   
-9222804379051090963    0    1  34.366476  53.679924  46.349121  45.043467   
-9222547355321442980    0    1  34.366476  53.679924  46.349121  45.043467   
...                   ...  ...        ...        ...        ...        ...   
 9220025367096517589    0    1  29.710450  49.049067  35.497220  44.983512   
 9221137650467601654    1    0  61.733254  55.062245  49.730006  52.791626   
 9221253244137085517    0    1 

In [4]:
datasets = split_imputed_dataframe(df, 0.1, 0.8, constants, generator=rng)

In [3]:
chunk_size = 1_000_000
selected_columns = [('TARGET', col) for col in constants['target_columns']] + [('AUTHOR', 'AUTHOR')]

# Initialize an empty DataFrame to store the selected columns
original = pd.DataFrame()

for chunk in pd.read_csv(paths['new']['w-emoji'], header=[0, 1], index_col=0, chunksize=chunk_size):
    filtered_chunk = chunk[selected_columns]
    original = pd.concat([original, filtered_chunk])

In [4]:
original.columns = original.columns.droplevel(0)

In [5]:
original = original.groupby('AUTHOR').mean()

In [6]:
def multiindex_imp(data:pd.DataFrame, constants:dict) -> pd.DataFrame:
    stat_list = [('STATS', stat) for stat in constants['stats_columns']]
    target_list = [('IMP_TARGET', target) for target in constants["target_columns"]]
    tuples =  stat_list + target_list
    multiindex = pd.MultiIndex.from_tuples(tuples, names=['GROUP', 'FEATURE'])
    data_multiindexed = pd.DataFrame(columns=multiindex)
    data_multiindexed['STATS'] = data[constants['stats_columns']]
    data_multiindexed['IMP_TARGET'] = data[constants['target_columns']]
    return data_multiindexed

def multiindex_org(data:pd.DataFrame, constants:dict) -> pd.DataFrame:
    target_list = [('ORG_TARGET', target) for target in constants["target_columns"]]
    multiindex = pd.MultiIndex.from_tuples(target_list, names=['GROUP', 'FEATURE'])
    data_multiindexed = pd.DataFrame(columns=multiindex)
    data_multiindexed['ORG_TARGET'] = data[constants['target_columns']]
    return data_multiindexed

In [7]:
original

FEATURE,mbtiEXT,mbtiSEN,mbtiTHI,mbtiJUD,cEXT,cNEU,cAGR,cCON,cOPN,sEXT,sNEU,sAGR,sCON,sOPN
AUTHOR,,,,,,,,,,,,,,
-9223192462986286358,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9223027331140286803,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222856035137654294,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222804379051090963,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
-9222547355321442980,0.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9220025367096517589,0.0,0.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9221137650467601654,1.0,0.0,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9221253244137085517,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
imputed

,chars,uppercased,emojis,posts,duplicates,word_nonwords,nonword_words,nonword_spaces,space_punctuations,hashtags,...,cEXT,cNEU,cAGR,cCON,cOPN,sEXT,sNEU,sAGR,sCON,sOPN
0,67.496063,4.929134,0.850394,127.0,0.157480,2.992126,2.212598,0.622047,0.007874,0.299213,...,0.739319,0.053549,1.000000,0.669351,1.000000,49.297890,36.716946,45.335832,63.447506,46.241635
1,169.280000,6.600000,0.000000,50.0,0.840000,5.680000,1.940000,1.460000,0.200000,0.000000,...,0.267549,0.000000,0.378116,0.248706,0.884025,60.705837,23.922539,58.315340,47.285805,77.420998
2,122.727273,6.636364,0.000000,33.0,0.333333,4.060606,1.363636,0.545455,0.000000,0.000000,...,0.125044,0.419457,0.622937,0.292483,0.747981,52.911221,37.056990,66.238219,34.427927,45.667059
3,157.413043,6.956522,0.000000,46.0,0.934783,5.108696,1.847826,0.717391,0.195652,0.000000,...,0.013664,0.006736,0.027517,0.015736,0.986336,38.128597,8.457520,25.353986,45.162375,57.683056
4,154.255319,5.063830,0.000000,47.0,0.744681,5.255319,1.872340,0.191489,0.000000,0.000000,...,0.063860,0.034127,0.132830,0.084893,0.936140,36.111359,20.999387,44.670598,54.229267,48.340938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29491,68.147368,13.684211,0.957895,95.0,0.357895,2.484211,2.589474,0.136842,0.021053,0.189474,...,0.424709,0.165504,0.424709,0.670288,0.754420,45.381350,47.273300,52.387190,56.118723,52.979897
29492,164.494331,5.718821,0.000378,2646.0,0.134921,5.421013,2.487150,0.082011,0.001512,0.002268,...,0.203177,0.400553,0.399731,0.000000,0.802230,17.267649,32.966281,32.659386,38.227142,79.654052
29493,140.621622,3.081081,0.000000,37.0,0.783784,3.270270,1.270270,0.216216,0.027027,0.027027,...,0.088506,0.182786,0.911494,0.803453,0.911494,30.078067,47.276576,34.225521,61.661926,80.520540
29494,236.614375,9.807286,0.003917,5106.0,0.135527,8.597336,4.169800,0.356639,0.000979,0.001175,...,0.201577,0.400306,0.399838,0.000000,0.801080,24.415086,39.876178,47.931401,79.772492,26.952100


In [9]:
imputed = multiindex_imp(imputed, constants)

In [11]:
original = multiindex_org(original, constants)

In [18]:
orignial = original.reset_index()

In [19]:
data = orignial.join(imputed)

In [20]:
data.head()

GROUP                 AUTHOR ORG_TARGET                                    \
FEATURE                         mbtiEXT mbtiSEN mbtiTHI mbtiJUD cEXT cNEU   
0       -9223192462986286358        0.0     0.0     0.0     1.0  NaN  NaN   
1       -9223027331140286803        0.0     0.0     0.0     0.0  NaN  NaN   
2       -9222856035137654294        0.0     0.0     0.0     0.0  NaN  NaN   
3       -9222804379051090963        0.0     0.0     0.0     1.0  NaN  NaN   
4       -9222547355321442980        0.0     0.0     0.0     1.0  NaN  NaN   

GROUP                   ... IMP_TARGET                                \
FEATURE cAGR cCON cOPN  ...       cEXT      cNEU      cAGR      cCON   
0        NaN  NaN  NaN  ...   0.739319  0.053549  1.000000  0.669351   
1        NaN  NaN  NaN  ...   0.267549  0.000000  0.378116  0.248706   
2        NaN  NaN  NaN  ...   0.125044  0.419457  0.622937  0.292483   
3        NaN  NaN  NaN  ...   0.013664  0.006736  0.027517  0.015736   
4        NaN  NaN  NaN  ...   0.063860  0.034127  0.132830  0.084893   

GROUP                                                                     
FEATURE      cOPN       sEXT       sNEU       sAGR       sCON       sOPN  
0        1.000000  49.297890  36.716946  45.335832  63.447506  46.241635  
1        0.884025  60.705837  23.922539  58.315340  47.285805  77.420998  
2        0.747981  52.911221  37.056990  66.238219  34.427927  45.667059  
3        0.986336  38.128597   8.457520  25.353986  45.162375  57.683056  
4        0.936140  36.111359  20.999387  44.670598  54.229267  48.340938  

[5 rows x 41 columns]

In [23]:
data['IMP_TARGET'].head(20)

FEATURE,mbtiEXT,mbtiSEN,mbtiTHI,mbtiJUD,cEXT,cNEU,cAGR,cCON,cOPN,sEXT,sNEU,sAGR,sCON,sOPN
0,0.0,0.0,0.0,1.0,0.739319,0.053549,1.000000,0.669351,1.000000,49.297890,36.716946,45.335832,63.447506,46.241635
1,0.0,0.0,0.0,0.0,0.267549,0.000000,0.378116,0.248706,0.884025,60.705837,23.922539,58.315340,47.285805,77.420998
2,0.0,0.0,0.0,0.0,0.125044,0.419457,0.622937,0.292483,0.747981,52.911221,37.056990,66.238219,34.427927,45.667059
3,0.0,0.0,0.0,1.0,0.013664,0.006736,0.027517,0.015736,0.986336,38.128597,8.457520,25.353986,45.162375,57.683056
4,0.0,0.0,0.0,1.0,0.063860,0.034127,0.132830,0.084893,0.936140,36.111359,20.999387,44.670598,54.229267,48.340938
5,1.0,0.0,0.0,1.0,0.558720,0.330107,1.000000,0.000000,1.000000,59.796582,58.115493,72.723706,32.689940,82.763569
6,0.0,0.0,0.0,0.0,0.071323,0.000000,0.143076,0.062269,0.928677,55.411358,15.109995,38.707086,48.379183,61.662337
7,0.0,0.0,1.0,1.0,0.166219,0.160417,0.593411,0.418560,1.000000,60.778242,19.645444,42.314885,55.694617,83.142999
8,1.0,0.0,1.0,1.0,0.142532,0.343082,0.432255,0.000000,1.000000,35.080290,43.586163,34.295884,24.458539,48.010283
9,0.0,0.0,0.0,0.0,0.224037,0.124573,0.875427,0.263380,0.875427,37.220649,34.041351,35.966321,20.682211,72.496403
